In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq 

In [2]:
!sudo add-apt-repository ppa:webupd8team/java
!sudo apt-get update
!sudo apt-get install oracle-java8-installer

 The Oracle JDK License has changed for releases starting April 16, 2019.

The new Oracle Technology Network License Agreement for Oracle Java SE is substantially different from prior Oracle JDK licenses. The new license permits certain uses, such as personal use and development use, at no cost -- but other uses authorized under prior Oracle JDK licenses may no longer be available. Please review the terms carefully before downloading and using this product. An FAQ is available here: https://www.oracle.com/technetwork/java/javase/overview/oracle-jdk-faqs.html

Oracle Java downloads now require logging in to an Oracle account to download Java updates, like the latest Oracle Java 8u211 / Java SE 8u212. Because of this I cannot update the PPA with the latest Java (and the old links were broken by Oracle).

For this reason, THIS PPA IS DISCONTINUED.

UPDATE:

For Oracle Java 17, see a different PPA -> https://www.linuxuprising.com/2021/09/how-to-install-oracle-java-17-lts-on.html

Old descr

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/NSL-KDD/

/content/drive/My Drive/NSL-KDD


In [87]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [88]:

spark = SparkSession.builder \
      .master("local") \
      .appName("NSL-KDD") \
      .config("spark.executor.memory", "12gb") \
      .getOrCreate()
sc = spark.sparkContext

In [89]:
sc

<SparkContext master=local appName=NSL-KDD>

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn as sk
import warnings


from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from pyspark.ml.feature import  VectorAssembler, OneHotEncoder, StringIndexer,StandardScaler, Normalizer



In [130]:
train_set=pd.read_csv("kdd_train.csv")

In [131]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [132]:
train_set.head(3)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.0,0.0,0.0,0.05,0.0,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.0,0.0,0.0,0.00,0.0,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.0,1.0,1.0,0.00,0.0,neptune


In [133]:
print("Train set dimension: ", train_set.shape)

Train set dimension:  (125973, 42)


In [134]:
print("------Train Set - Labels------")
print(train_set['labels'].value_counts())

------Train Set - Labels------
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: labels, dtype: int64


In [135]:
attack_or_not=[]
for i in train_set["labels"]:
    if i =="normal":
        attack_or_not.append("normal")
    else:
        attack_or_not.append("attack")           
train_set["labels"]=attack_or_not

In [136]:
train_set.labels.value_counts()

normal    67343
attack    58630
Name: labels, dtype: int64

In [137]:
train_set.iloc[:,-1] = LabelEncoder().fit_transform(train_set.iloc[:,-1])

In [138]:
dataset = spark.createDataFrame(data = train_set)

In [142]:
from pyspark.ml import Pipeline
categoricalColumns = ['protocol_type', 'service', 'flag']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + "_index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "_code"])
    stages += [stringIndexer, encoder]  

In [145]:
pipeline = Pipeline(stages = stages)
dataset = pipeline.fit(dataset).transform(dataset)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
x= dataset.select("labels")
y= dataset.drop("protocol_type","protocol_type_Index","flag","flag_Index","service","service_Index", "labels")

In [ ]:
assembler = VectorAssembler(inputCols = y.columns, outputCol = 'assembled_features')
y = assembler.transform(y)

In [ ]:
normalizedData=Normalizer().setP(1).setInputCol("assembled_features").setOutputCol("features")
y=normalizedData.transform(y)

In [ ]:
y = y.select("features")

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

y = y.withColumn("id", monotonically_increasing_id())
x = x.withColumn("id", monotonically_increasing_id())
last_dataset = y.join(x, y.id == x.id, how ="inner").drop("id")


In [ ]:
last_dataset.toPandas()

In [ ]:
train_data,test_data =last_dataset.randomSplit([0.7,0.3], seed=0)

In [ ]:
#**********************RANDOM FOREST ALGORİTMASI*******************

from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='labels', 
                            featuresCol='features',
                            maxDepth=5)
model = rf.fit(train_data)
predictions = model.transform(test_data)